In [78]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

from tqdm import tqdm
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamelog

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, ElasticNet, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

pd.options.display.max_columns=200

In [18]:
seasons = ["200{}-0{}".format(x, x+1) if x!=9 else "200{}-{}".format(x, x+1) for x in range(0, 10)]
seasons2 = ["20{}-{}".format(x, x+1) for x in range(10, 21)]
seasons.extend(seasons2)
print(seasons)

def gather_data(seasons = seasons):
    full_player_data = []
    for season in seasons:
        player_gls = pd.read_csv('./data/player_gamelogs_{}.csv'.format(season), dtype={'GAME_ID':'int'})
        player_gls_adv = pd.read_csv('./data/player_advanced_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'int'})
        player_gls_scoring = pd.read_csv('./data/player_scoring_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'int'})
#         player_gls_tracking = pd.read_csv('./data/player_tracking_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'object'})
    
        print("player_gls_shape:", player_gls.shape,
             "player_gls_adv_shape:", player_gls_adv.shape,
             "player_gls_scoring_shape:", player_gls_scoring.shape)
    
        player_gls = player_gls[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']]

        player_full = pd.merge(player_gls, player_gls_adv, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
        player_full = pd.merge(player_full, player_gls_scoring, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
    
        print("after_merging shape", player_full.shape)
        
        # drop unnecessary columns
        player_full = player_full.drop(columns = ['TEAM_ID', 'TEAM_NAME', 'TEAM_ID_x',
                         'TEAM_ABBREVIATION_x','TEAM_CITY','PLAYER_NAME_x','START_POSITION',
                          'COMMENT','MIN_x','TEAM_ID_x', 'TEAM_ABBREVIATION_x','TEAM_CITY_x', 
                          'PLAYER_NAME_x', 'START_POSITION_x', 'COMMENT_x', 'MIN_x', 
                         'TEAM_ID_x', 'TEAM_ABBREVIATION_x', 'TEAM_CITY_x', 'PLAYER_NAME_x', 
                          'START_POSITION_x', 'COMMENT_x', 'MIN_x', 'AST_x', 'FG_PCT_x',
                         'FG_PCT', 'FG3_PCT', 'FT_PCT', 'E_NET_RATING', 'NET_RATING', 
                         'AST_PCT', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'EFG_PCT', 'TS_PCT', 
                         'PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_FT', 'CFG_PCT',
                        'UFG_PCT', 'DFG_PCT'], 
                       errors= 'ignore')

        # Convert date to datetime
        player_full['GAME_DATE'] = pd.to_datetime(player_full['GAME_DATE'])
        
        # Convert W/L to a binary 1/0 for win/loss
        player_full['WL'] = (player_full['WL'].str[0] == 'W').astype(int)
        
        # Add a binary home_game feature
        player_full['HOME_GAME'] = 0 
        player_full['HOME_GAME'] = (player_full['MATCHUP'].str[:6] == player_full['TEAM_ABBREVIATION'].str[:] + ' vs').astype(int)
        
        # convert the percentages into raw numbers (because we will be averaging them, we don't want to average percentages)
        # for example, percentage of points from midrange will be points scored from midrange
        player_full['FG2M'] = player_full['FGM'] - player_full['FG3M']
        player_full['FG2A'] = player_full['FGA'] - player_full['FG3A']
        player_full['PTS_2PT'] = (player_full['PTS'] * player_full['PCT_PTS_2PT']).astype('int8')
        player_full['PTS_2PT_MR'] = (player_full['PTS'] * player_full['PCT_PTS_2PT_MR']).astype('int8')
        player_full['PTS_3PT'] = (player_full['PTS'] * player_full['PCT_PTS_3PT']).astype('int8')
        player_full['PTS_FB'] = (player_full['PTS'] * player_full['PCT_PTS_FB']).astype('int8')
        player_full['PTS_OFF_TOV'] = (player_full['PTS'] * player_full['PCT_PTS_OFF_TOV']).astype('int8')
        player_full['PTS_PAINT'] = (player_full['PTS'] * player_full['PCT_PTS_PAINT']).astype('int8')
        player_full['AST_2PM'] = (player_full['FG2M'] * player_full['PCT_AST_2PM']).astype('int8')
        player_full['UAST_2PM'] = (player_full['FG2M'] * player_full['PCT_UAST_2PM']).astype('int8')
        player_full['AST_3PM'] = (player_full['FG3M'] * player_full['PCT_AST_3PM']).astype('int8')
        player_full['UAST_3PM'] = (player_full['FG3M'] * player_full['PCT_UAST_3PM']).astype('int8')



        player_full = player_full.drop(columns = ['PCT_PTS_2PT', 'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 
                                                  'PCT_PTS_FB', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT',
                                                  'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM',
                                                  'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM', 
                                                  'FGM', 'FGA', 'AST_TOV', 'USG_PCT', 'PACE',
                                                  'PACE_PER40', ], errors='ignore')
        
        
        full_player_data.append(player_full)
        
        
    
    player_df = pd.concat(full_player_data)
    
    
    player_df['home_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 1 else row['MATCHUP'][-3:], axis=1)
    player_df['away_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 0 else row['MATCHUP'][-3:], axis=1)
    
    player_df[['home_team_abbr', 'away_team_abbr']] = player_df[['home_team_abbr', 'away_team_abbr']].replace({'NOH':'NOP',
                                                                                                               'NJN':'BKN'})
    
    # Reorder columns
    player_df = player_df[['SEASON_YEAR', 'TEAM_ABBREVIATION', 'PLAYER_NAME', 'PLAYER_ID', 
                           'home_team_abbr', 'away_team_abbr', 'GAME_ID', 
                           'GAME_DATE', 'MATCHUP', 'WL', 'HOME_GAME', 'MIN',
                           'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
                           'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
                           'PTS', 'PLUS_MINUS', 'E_OFF_RATING', 'E_DEF_RATING',
                           'AST_RATIO', 'TM_TOV_PCT', 'E_USG_PCT',
                           'E_PACE', 'POSS', 'PIE',
                           'FG2M', 'FG2A', 'PTS_2PT_MR', 'PTS_FB', 
                           'PTS_OFF_TOV', 'PTS_PAINT', 'AST_2PM',
                           'UAST_2PM', 'AST_3PM', 'UAST_3PM']]
    
    return player_df

player_df = gather_data(seasons)

['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21']
player_gls_shape: (25153, 64) player_gls_adv_shape: (31419, 31) player_gls_scoring_shape: (29475, 24)
after_merging shape (25153, 83)
player_gls_shape: (20758, 64) player_gls_adv_shape: (27638, 31) player_gls_scoring_shape: (25455, 24)
after_merging shape (20758, 83)
player_gls_shape: (25757, 64) player_gls_adv_shape: (33542, 31) player_gls_scoring_shape: (31338, 24)
after_merging shape (25757, 83)
player_gls_shape: (25618, 64) player_gls_adv_shape: (33681, 31) player_gls_scoring_shape: (31378, 24)
after_merging shape (25618, 83)
player_gls_shape: (25981, 64) player_gls_adv_shape: (33522, 31) player_gls_scoring_shape: (31412, 24)
after_merging shape (25981, 83)
player_gls_shape: (26078, 64) player_gls_adv_shape: (33659, 31) player_gls_scoring_shape: (31423, 24)
after_merging shape (26078, 83)
player_gls_shape: (26139, 64) player_gls_adv_shape: (33610, 31) player_gls

In [88]:
player_df

,SEASON_YEAR,TEAM_ABBREVIATION,PLAYER_NAME,PLAYER_ID,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,E_DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,FG2M,FG2A,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
0,2010-11,MIL,Corey Maggette,1894,OKC,MIL,21001225,2011-04-13,MIL @ OKC,1,0,18.776667,0,0,6,6,0,1,2,2,1,0,0,2,3,12,2,111.0,105.4,16.7,16.7,0.239,96.89,39,0.139,3,5,3,2,0,2,3,0,0,0
1,2010-11,NOH,Jarrett Jack,101127,DAL,NOP,21001223,2011-04-13,NOH @ DAL,0,0,28.658333,1,1,5,6,0,0,7,1,0,1,0,2,4,22,-22,100.3,137.0,28.0,4.0,0.296,95.91,56,0.144,7,14,6,2,0,8,2,4,1,0
2,2010-11,NJN,Jordan Farmar,200770,CHI,BKN,21001222,2011-04-13,NJN @ CHI,0,0,38.166667,2,6,5,6,0,4,12,4,0,2,0,1,4,21,7,107.9,102.8,35.3,11.8,0.274,91.28,71,0.198,5,9,3,1,6,6,0,5,1,1
3,2010-11,NYK,Amar'e Stoudemire,2405,BOS,NYK,21001216,2011-04-13,NYK @ BOS,0,0,20.450000,0,0,2,3,2,2,1,2,0,1,0,1,4,14,-3,107.6,107.5,5.3,10.5,0.384,101.49,40,0.072,6,15,4,0,0,7,1,4,0,0
4,2010-11,OKC,Byron Mullens,201957,OKC,MIL,21001225,2011-04-13,OKC vs. MIL,0,1,29.178333,0,0,2,4,1,4,0,2,1,0,1,2,2,10,-22,90.1,130.8,0.0,15.4,0.191,97.12,60,0.041,4,9,0,1,4,8,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8689,2020-21,GSW,Jordan Poole,1629673,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,17.916667,0,1,0,0,0,2,3,2,0,0,0,2,1,2,-3,83.8,94.8,33.3,22.2,0.135,103.63,38,0.000,1,3,0,0,0,2,0,1,0,0
8690,2020-21,GSW,Andrew Wiggins,203952,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,31.233333,2,6,3,4,0,2,1,4,0,1,2,4,4,13,-28,74.4,115.8,4.3,17.4,0.262,111.88,72,-0.044,2,10,2,0,0,2,1,1,2,0
8691,2020-21,GSW,Juan Toscano-Anderson,1629308,BKN,GSW,22000001,2020-12-22,GSW @ BKN,0,0,13.250000,0,0,0,0,0,4,2,0,0,2,0,3,1,4,-9,70.5,95.5,50.0,0.0,0.058,119.84,33,0.208,2,2,0,2,0,4,2,0,0,0
8692,2020-21,LAC,Luke Kennard,1628379,LAL,LAC,22000002,2020-12-22,LAC @ LAL,1,0,21.055000,0,3,0,0,0,2,2,1,2,1,0,3,0,4,-3,116.9,112.2,22.2,11.1,0.144,104.59,44,0.032,2,3,0,0,0,4,1,1,0,0


In [292]:
def aggregate_player_stats(df = player_df):

    seasons = player_df['SEASON_YEAR'].unique()

    df = df.sort_values('GAME_DATE')

    for season in seasons:
        season_df = df.loc[df['SEASON_YEAR'] == season]
        player_ids = season_df['PLAYER_ID'].unique()
        for p_id in tqdm(player_ids, desc='Progress'):
            player_log = season_df.loc[season_df['PLAYER_ID'] == p_id]

            avg_player_data = player_log.iloc[:, 11:].shift(1).rolling(10, min_periods=5).mean()

            cols = avg_player_data.columns

            df.loc[(df['SEASON_YEAR'] == season) & (df['PLAYER_ID'] == p_id), cols] = avg_player_data

    df.to_csv('player_avg_last10.csv', index=False)
        
    return df
    
player_stats = aggregate_player_stats(df=player_df)

Progress: 100%|██████████████████████████████████████████████████████████████████████| 480/480 [00:52<00:00,  9.07it/s]


In [293]:
player_stats.loc[player_stats['GAME_ID']==21000075].sort_values('MIN', ascending=False)

,SEASON_YEAR,TEAM_ABBREVIATION,PLAYER_NAME,PLAYER_ID,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,E_DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,FG2M,FG2A,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
23603,2010-11,MIA,LeBron James,2544,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,34.323333,1.0,3.4,6.2,8.0,0.0,4.6,7.2,5.2,1.4,0.8,0.0,1.8,4.4,20.4,14.0,109.96,83.66,24.92,16.64,0.3224,94.648,66.8,0.1922,5.6,10.6,3.8,4.6,4.8,7.2,1.8,3.4,1.0,0.0
23527,2010-11,MIA,Chris Bosh,2547,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,31.736667,0.0,0.2,3.8,4.6,1.0,5.4,1.6,0.8,1.0,1.0,0.8,2.2,2.8,13.0,15.0,117.26,87.02,10.52,4.70,0.2164,92.952,60.6,0.1264,4.6,10.6,4.0,0.0,2.0,4.6,2.6,1.6,0.0,0.0
23563,2010-11,MIA,Dwyane Wade,2548,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,31.570667,1.0,2.6,5.8,8.0,1.2,3.8,4.2,3.0,1.6,1.0,1.0,2.8,5.4,22.4,10.8,113.96,90.06,15.52,10.78,0.3498,93.192,60.0,0.1774,6.8,13.8,1.6,4.8,5.6,11.6,1.8,4.2,0.2,0.8
23730,2010-11,MIA,James Jones,2592,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,27.129000,3.6,6.8,0.0,0.0,0.2,2.2,0.8,0.0,0.6,0.4,0.0,2.0,1.2,11.6,6.2,105.40,91.18,10.04,0.00,0.1364,94.588,53.6,0.1214,0.4,1.0,0.8,1.0,2.4,0.0,0.4,0.0,3.6,0.0
23685,2010-11,MIA,Carlos Arroyo,2306,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,23.359667,0.4,0.8,1.2,1.4,0.4,3.2,2.4,1.0,0.6,0.0,0.0,1.4,0.8,7.2,13.4,111.90,82.10,23.26,9.62,0.1394,91.746,43.8,0.1214,2.4,4.6,4.2,0.2,0.8,0.2,1.4,0.6,0.4,0.0
23585,2010-11,MIA,Udonis Haslem,2617,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,22.872000,0.0,0.0,1.6,2.0,1.8,7.0,0.2,0.8,0.4,0.4,0.4,3.6,2.4,7.2,2.8,98.20,91.04,2.22,12.52,0.1510,96.178,45.8,0.1274,2.8,5.8,3.6,0.8,1.0,1.6,2.4,0.4,0.0,0.0
23712,2010-11,MIA,Eddie House,2067,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,22.689667,2.0,3.2,1.0,1.2,0.0,1.8,1.8,0.8,1.4,0.0,0.2,1.4,0.6,9.0,-0.2,98.72,98.40,19.54,7.78,0.1596,93.754,44.4,0.1304,1.0,3.4,1.8,1.0,0.6,0.0,0.6,0.4,1.8,0.2
23518,2010-11,MIA,Joel Anthony,201202,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,20.329333,0.0,0.0,0.6,1.2,1.2,3.2,0.6,0.4,0.4,1.4,0.0,3.4,1.0,1.4,7.8,101.32,76.84,20.00,30.00,0.0406,91.318,37.6,0.0540,0.4,0.6,0.0,0.4,0.2,0.8,0.2,0.2,0.0,0.0
23728,2010-11,MIA,Zydrunas Ilgauskas,980,NOP,MIA,21000075,2010-11-05,MIA @ NOH,0,0,14.649667,0.0,0.0,0.8,1.2,2.2,2.0,0.6,0.6,0.6,0.8,0.0,3.6,1.0,6.0,10.2,127.04,85.52,7.30,15.56,0.1900,97.444,29.8,0.0874,2.6,4.8,3.6,0.0,0.6,1.4,2.0,0.4,0.0,0.0
23581,2010-11,NOH,Chris Paul,101108,NOP,MIA,21000075,2010-11-05,NOH vs. MIA,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [750]:
player_stats1 = pd.read_csv('player_l10_avg_part1.csv', parse_dates=['GAME_DATE'])
player_stats2 = pd.read_csv('player_l10_avg_part2.csv', parse_dates=['GAME_DATE'])

player_stats = pd.concat([player_stats1, player_stats2])

In [751]:
player_stats = player_stats.sort_values('GAME_DATE')
player_stats

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,HOME_GAME,WL,MIN,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,E_DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,PACE_PER40,POSS,PIE,PTS_2PT,PTS_2PT_MR,PTS_3PT,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
0,2000-01,393,Rod Strickland,WAS,WAS,BKN,20000063,2000-11-08,WAS vs. NJN,1,0,33.932000,5.2,11.2,0.0,0.2,5.0,6.8,0.4,3.4,7.2,2.0,1.8,0.4,1.0,1.4,0.0,15.4,3.2,94.30,91.38,29.20,8.78,0.2196,79.114,67.0,0.1900,10.4014,4.3968,0.000000e+00,3.4004,1.9990,5.9968,2.4006,2.7994,0.0000,0.0000
19,2000-01,1517,Bobby Jackson,SAC,SAC,GSW,20000071,2000-11-08,SAC vs. GSW,1,1,37.470000,3.2,7.6,0.4,2.2,3.8,4.4,0.8,4.4,4.4,2.2,2.0,0.2,0.4,2.6,0.0,11.4,0.0,93.02,92.42,26.64,10.76,0.1636,77.442,72.8,0.1114,6.3984,2.8004,1.199800e+00,4.2000,4.2000,3.6008,0.9996,2.2004,0.2000,0.2000
20,2000-01,98,Nick Anderson,SAC,SAC,GSW,20000071,2000-11-08,SAC vs. GSW,1,1,7.110000,0.0,0.6,0.6,1.0,0.0,0.0,0.4,0.6,0.0,0.2,0.6,0.2,0.4,0.2,0.0,1.8,2.0,101.58,92.32,0.00,20.00,0.1124,77.998,14.2,0.1032,0.0000,0.0000,1.800000e+00,0.0000,0.0000,0.0000,0.0000,0.0000,0.6000,0.0000
21,2000-01,1727,Pat Garrity,ORL,ORL,CHH,20000064,2000-11-08,ORL vs. CHH,1,0,23.225333,2.2,4.8,1.0,3.8,1.4,1.4,0.2,1.6,1.0,1.0,0.6,0.2,0.2,3.0,0.0,8.8,0.6,91.72,94.00,9.04,7.66,0.1870,82.008,47.2,0.0342,4.4022,3.1980,2.999800e+00,0.6000,2.2018,1.1986,0.9998,1.2002,1.0000,0.0000
22,2000-01,270,Horace Grant,LAL,SAS,LAL,20000068,2000-11-08,LAL @ SAS,0,0,36.589333,2.8,6.4,0.0,0.2,1.4,2.0,3.6,4.8,0.8,0.6,0.2,0.4,0.2,3.4,0.0,7.0,5.2,100.00,94.52,6.82,4.60,0.1020,75.982,69.4,0.0592,5.6006,1.6018,0.000000e+00,0.4000,0.4000,3.9988,2.3998,0.4002,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244706,2020-21,1628389,Bam Adebayo,MIA,SAC,MIA,22000447,2021-02-18,MIA @ SAC,0,1,35.135167,7.5,13.3,0.0,0.0,4.7,5.8,2.2,7.2,5.0,2.8,1.3,1.4,0.9,2.9,5.8,19.7,0.0,108.30,107.61,20.64,11.56,0.2393,81.570,71.7,0.1614,14.9998,1.9981,4.440892e-17,1.6002,2.9999,13.0017,4.7002,2.7998,0.0000,0.0000
244707,2020-21,1629130,Duncan Robinson,MIA,SAC,MIA,22000447,2021-02-18,MIA @ SAC,0,1,31.558833,0.7,1.1,3.0,8.3,0.6,0.8,0.0,3.3,1.2,1.3,0.4,0.1,0.0,2.9,0.9,11.0,1.2,103.91,102.29,9.17,10.24,0.1610,80.680,63.5,0.0523,1.4010,0.0000,8.999000e+00,1.1995,1.6001,1.4010,0.6000,0.1000,3.0000,0.0000
244708,2020-21,1630169,Tyrese Haliburton,SAC,SAC,MIA,22000447,2021-02-18,SAC vs. MIA,1,0,31.138667,3.0,4.8,2.6,6.1,0.5,0.6,1.3,3.2,5.6,1.8,1.3,0.4,0.1,1.2,0.7,14.3,0.7,114.08,114.59,30.09,9.38,0.1791,85.265,66.8,0.1232,5.9998,1.4000,7.800000e+00,1.7998,1.4007,4.6020,0.9998,2.0002,2.3001,0.2999
244694,2020-21,203507,Giannis Antetokounmpo,MIL,MIL,TOR,22000445,2021-02-18,MIL vs. TOR,1,0,34.527833,9.9,15.3,0.8,2.9,7.2,10.0,1.7,10.4,6.3,3.1,1.5,1.7,0.9,2.5,8.3,29.4,6.7,119.39,108.35,20.76,9.79,0.3133,86.329,74.1,0.2069,19.7979,0.8029,2.397400e+00,5.9008,4.4984,19.0030,4.2016,5.6984,0.1000,0.7000


In [768]:
set(player_stats['GAME_ID'].unique()) - set(all_team_gamelogs['GAME_ID'].unique()) 

{22000427,
 22000429,
 22000430,
 22000431,
 22000432,
 22000433,
 22000436,
 22000437,
 22000438,
 22000439,
 22000440,
 22000441,
 22000443,
 22000444,
 22000445,
 22000446,
 22000447,
 22000562,
 22000564}

In [864]:
def prep_data_for_model(df = player_stats):
    matchup_info = []
    matchups = []
    targets = []
    
    all_gamelogs = []
    for season in seasons:
        gl = pd.read_csv("./data/basic_team_boxscores/team_gamelogs_{}.csv".format(season))
        all_gamelogs.append(gl)

    all_team_gamelogs = pd.concat(all_gamelogs)
    game_results = all_team_gamelogs.loc[all_team_gamelogs['MATCHUP'].str.contains('vs'), ['GAME_ID', 'PLUS_MINUS']]

    game_ids = set(player_stats['GAME_ID'].unique()) & set(all_team_gamelogs['GAME_ID'].unique()) 
    
    for game_id in tqdm(game_ids, desc="progress"):
        game_df = df.loc[df['GAME_ID'] == game_id]
   
        home = game_df.loc[game_df['HOME_GAME'] == 1].sort_values('MIN', ascending=False)
        home_stats = home.iloc[:10, 11:].values
        
        away = game_df.loc[game_df['HOME_GAME'] == 0].sort_values('MIN', ascending=False)
        away_stats = away.iloc[:10, 11:].values
                
        # If a team had one of their game's removed (because it was in the first 5 games of the season), remove the entire game
        if (home_stats.shape[0] == 0) or (away_stats.shape[0] == 0):
            continue
        
        if home_stats.shape[0] < 10:
            missing_rows = 10 - home_stats.shape[0]
            padding = np.zeros((missing_rows, home_stats.shape[1]))
            home_stats = np.concatenate([home_stats, padding], axis=0)
        
        if away_stats.shape[0] < 10:
            missing_rows = 10 - away_stats.shape[0]
            padding = np.zeros((missing_rows, away_stats.shape[1]))
            away_stats = np.concatenate([away_stats, padding], axis=0)

        matchup = np.stack([home_stats, away_stats], axis=0)
        matchups.append(matchup)
        
        point_diff = game_results.loc[game_results['GAME_ID'] == game_id, 'PLUS_MINUS'].values[0]
        targets.append(point_diff)
        
        matchup_info.append(home[['SEASON_YEAR', 'home_team_abbr', 'away_team_abbr', 'GAME_ID', 'GAME_DATE', 'WL']].iloc[0].values)
        
    matchup_features = np.stack(matchups, axis=0)
    matchup_info = pd.DataFrame(matchup_info, columns=['SEASON_YEAR', 'home_team_abbr', 'away_team_abbr', 'GAME_ID', 'GAME_DATE', 'WL'])
    
    return matchup_info, matchup_features, np.array(targets).reshape(-1, 1)

matchup_info, matchup_features, targets = prep_data_for_model(df = player_stats)

progress: 100%|█████████████████████████████████████████████████████████████████| 24611/24611 [02:04<00:00, 198.12it/s]


In [865]:
matchup_info.shape, matchup_features.shape, targets.shape

((24415, 6), (24415, 2, 10, 36), (24415, 1))

In [772]:
pd.DataFrame(matchup_info, columns=['SEASON_YEAR', 'home_team_abbr', 'away_team_abbr', 'GAME_ID', 'GAME_DATE', 'WL'])

,SEASON_YEAR,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,WL
0,2015-16,CHI,OKC,21500070,2015-11-05,1
1,2015-16,POR,MEM,21500074,2015-11-05,1
2,2015-16,ORL,TOR,21500075,2015-11-06,1
3,2015-16,NYK,MIL,21500079,2015-11-06,0
4,2015-16,IND,MIA,21500080,2015-11-06,1
...,...,...,...,...,...,...
24410,2005-06,DAL,MIA,40500402,2006-06-11,1
24411,2005-06,MIA,DAL,40500403,2006-06-13,1
24412,2005-06,MIA,DAL,40500404,2006-06-15,1
24413,2005-06,MIA,DAL,40500405,2006-06-18,1


In [774]:
home_cols = ["home_"+x for x in player_stats.columns[11:]]
away_cols = ["away_"+x for x in player_stats.columns[11:]]

cols = home_cols
pd.DataFrame(matchup_features[1011][0], columns=cols)


,home_MIN,home_FG2M,home_FG2A,home_FG3M,home_FG3A,home_FTM,home_FTA,home_OREB,home_DREB,home_AST,home_TOV,home_STL,home_BLK,home_BLKA,home_PF,home_PFD,home_PTS,home_PLUS_MINUS,home_E_OFF_RATING,home_E_DEF_RATING,home_AST_RATIO,home_TM_TOV_PCT,home_E_USG_PCT,home_PACE_PER40,home_POSS,home_PIE,home_PTS_2PT,home_PTS_2PT_MR,home_PTS_3PT,home_PTS_FB,home_PTS_OFF_TOV,home_PTS_PAINT,home_AST_2PM,home_UAST_2PM,home_AST_3PM,home_UAST_3PM
0,38.120667,6.7,12.7,1.6,6.3,4.6,5.4,0.4,2.7,5.5,3.5,0.8,0.5,1.1,3.5,5.4,22.8,-7.2,98.07,108.68,17.96,11.58,0.2770,83.894,79.8,0.0994,13.4019,5.6007,4.8000,5.299600e+00,4.0025,7.8027,2.2000,4.5000,1.1001,0.4999
1,35.949833,3.8,9.6,2.8,8.0,2.2,2.5,1.1,3.9,4.7,3.9,0.9,0.7,1.0,2.7,3.6,18.2,-6.6,95.89,107.13,17.35,14.18,0.2633,84.516,76.0,0.0835,7.6012,1.7999,8.3993,3.700000e+00,2.7016,5.8013,1.1003,2.6997,1.6997,1.1003
2,32.035167,2.3,5.0,0.8,2.8,1.5,2.2,2.6,4.0,2.0,1.3,1.3,0.3,0.8,2.3,1.7,8.5,-2.8,98.37,104.93,15.93,11.49,0.1277,83.780,67.5,0.0630,4.6009,0.5995,2.4002,1.300200e+00,1.8015,4.0015,0.8998,1.4002,0.8000,0.0000
3,29.309167,4.1,11.9,0.0,0.2,3.5,4.6,2.8,7.5,2.5,2.8,0.3,0.7,1.6,3.3,3.8,11.7,-7.4,95.72,108.83,12.25,14.90,0.2430,82.971,60.7,0.0733,8.1982,3.0019,0.0000,4.440892e-17,1.2008,5.2018,3.2000,0.9000,0.0000,0.0000
4,24.606000,1.7,4.0,2.2,6.0,1.5,1.9,0.5,3.6,1.2,0.8,0.4,0.2,0.5,2.6,2.2,11.5,-2.8,108.15,111.60,10.78,5.63,0.2070,84.547,52.2,0.0801,3.4002,1.6013,6.5995,2.200900e+00,0.9998,1.7989,1.2001,0.4999,2.0000,0.2000
5,22.055000,0.7,1.4,1.3,3.7,0.3,0.8,0.5,1.5,2.7,0.7,0.5,0.0,0.1,2.5,1.2,5.6,-3.7,95.78,108.61,24.97,6.84,0.1171,83.561,44.7,0.0309,1.4007,0.2000,3.8999,4.004000e-01,0.7003,1.2007,0.0000,0.7000,1.2001,0.0999
6,16.334167,2.3,4.6,0.1,1.4,1.3,2.5,0.5,2.5,1.5,1.3,0.1,0.1,0.7,1.6,2.0,6.2,-5.8,94.64,110.04,14.02,13.99,0.2163,87.489,36.0,0.0617,4.6000,0.0000,0.3000,1.899800e+00,0.6998,4.6000,0.5999,1.7001,0.1000,0.0000
7,13.576667,1.3,2.9,0.7,1.4,0.6,0.8,0.4,1.7,1.1,0.9,0.0,0.0,0.3,0.4,0.5,5.3,-4.5,100.01,113.21,16.58,15.00,0.1976,85.339,29.4,0.0814,2.5997,1.5997,2.0998,8.001000e-01,0.4003,1.0000,0.9000,0.4000,0.7000,0.0000
8,11.629000,1.0,1.7,0.2,1.2,0.3,0.4,0.7,1.2,1.0,0.3,0.3,0.0,0.1,1.3,0.4,2.9,-3.3,102.53,117.99,17.72,16.11,0.1319,80.978,23.9,0.0638,2.0003,0.2000,0.6000,9.000000e-01,0.4998,1.8003,0.6000,0.4000,0.2000,0.0000
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0000,0.000,0.0,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [866]:
# Split up data - training set and testing set
X_train, X_test, y_train, y_test = train_test_split(matchup_features, targets, test_size=0.2)


print("X_train shape:", X_train.shape,
      "\nX_test shape:", X_test.shape,
      "\ny_test shape:", y_train.shape,
      "\ny_train shape:", y_test.shape)

X_train shape: (19532, 2, 10, 36) 
X_test shape: (4883, 2, 10, 36) 
y_test shape: (19532, 1) 
y_train shape: (4883, 1)


In [867]:
class PlayerBoxScoreMatchupsDataset(Dataset):
    """"""
    def __init__(self, matchup_features, targets):
        self.targets = targets
        self.matchup_data = matchup_features
        
    def __len__(self):
        return len(self.matchup_data)
    
    def __getitem__(self, index):
        X = self.matchup_data[index]
        y = self.targets[index]
        
        return X, y
    

In [868]:
X_train.shape

(19532, 2, 10, 36)

In [1038]:
# Hyperparameters
input_size = 72
learning_rate = 0.001
batch_size = 7
epochs = 100

# Create DataSet and DataLoaders

training_set = PlayerBoxScoreMatchupsDataset(X_train, y_train)
validation_set = PlayerBoxScoreMatchupsDataset(X_test, y_test)

train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)

data, target = next(iter(train_loader))
print(data.shape, target.shape)
# print(data, target)
# data = torch.rand(1, 2, 10, 34)
# data

torch.Size([7, 2, 10, 36]) torch.Size([7, 1])


In [1043]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv1d(2, 2, kernel_size=(10,1), stride=1)
#         self.conv1_bn = nn.BatchNorm2d(2)
        
        self.fc1 = nn.Linear(72, 32)
#         self.fc1_bn = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(32, 1)
# # #         self.fc2_bn = nn.BatchNorm1d(16)
#         self.fc3 = nn.Linear(32, 1)
        
#         self.dropout = nn.Dropout(0.1)
        
        
    def forward(self, x):
        
        x = F.tanh(self.conv1(x))
#         print("shape after conv1:", x.shape)
        x = x.reshape(-1, 72)
#         x = self.dropout(x)
        x = F.tanh(self.fc1(x))
#         x = self.dropout(x)
        x = self.fc2(x)
# # #         x = self.dropout(x)
#         x = self.fc3(x)
        
        return x
    

import torch.optim as optim

net = Net()
loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.03)

epochs = 100

for e in range(epochs):
    train_loss = 0
    val_loss = 0
    net.train()
#     for data, target in train_loader:
    optimizer.zero_grad()
    
    output = net(data.float())
#     print("output:", output)
#     print("target:", target)
    if e == 99:
#         print("input 1", data[0][0][0], "\ninput 2:", data[1][0][0])
        print("outputs:", output, "output shape:", output.shape)
        print("targets:", target, "target shape:", target.shape)
    loss = loss_function(output, target.float())
    if e == 99:
        print("loss:", loss)

    loss.backward()

    optimizer.step()
    train_loss += loss.item() * data.size(0)
        
#     net.eval()
#     for data, target in val_loader:
#         output = net(data.float())
#         loss = loss_function(output, target.float())
        
#         val_loss += loss.item()*data.size(0)
        
#     train_loss = train_loss / len(train_loader.sampler)
    
#     val_loss = val_loss / len(val_loader.sampler)

#     if e % 5 == 0:
#         print('Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f}'.format(
#     e, train_loss, val_loss))
    


outputs: tensor([[9.1365],
        [9.1365],
        [9.1365],
        [9.1365],
        [9.1365],
        [9.1365],
        [9.1365]], grad_fn=<AddmmBackward>) output shape: torch.Size([7, 1])
targets: tensor([[22],
        [ 1],
        [-7],
        [-2],
        [19],
        [ 8],
        [23]]) target shape: torch.Size([7, 1])
loss: tensor(129.5511, grad_fn=<MseLossBackward>)


In [1037]:
import torch.optim as optim
# torch.manual_seed(5)

net = Net()
loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.03)

epochs = 100

for e in range(epochs):
    train_loss = 0
    val_loss = 0
    net.train()
#     for data, target in train_loader:
    optimizer.zero_grad()
    
    output = net(data.float())
#     print("output:", output)
#     print("target:", target)
    if e == 99:
        print("input 1", data[0][0][0], "\ninput 2:", data[1][0][0])
        print("outputs:", output, "output shape:", output.shape)
        print("targets:", target, "target shape:", target.shape)
    loss = loss_function(output, target.float())
    if e == 99:
        print("loss:", loss)

    loss.backward()

    optimizer.step()
    train_loss += loss.item() * data.size(0)
        
#     net.eval()
#     for data, target in val_loader:
#         output = net(data.float())
#         loss = loss_function(output, target.float())
        
#         val_loss += loss.item()*data.size(0)
        
#     train_loss = train_loss / len(train_loader.sampler)
    
#     val_loss = val_loss / len(val_loader.sampler)

#     if e % 5 == 0:
#         print('Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f}'.format(
#     e, train_loss, val_loss))
    

input 1 tensor([4.0895e+01, 1.1400e+01, 2.1300e+01, 0.0000e+00, 1.0000e-01, 5.1000e+00,
        7.0000e+00, 2.2000e+00, 9.2000e+00, 3.5000e+00, 1.8000e+00, 1.3000e+00,
        4.4000e+00, 1.3000e+00, 2.5000e+00, 6.4000e+00, 2.7900e+01, 3.7000e+00,
        1.0660e+02, 1.0042e+02, 1.1810e+01, 5.9300e+00, 2.9630e-01, 7.7957e+01,
        7.9500e+01, 1.9220e-01, 2.2806e+01, 1.1203e+01, 0.0000e+00, 2.9036e+00,
        4.5028e+00, 1.1602e+01, 8.0989e+00, 3.3011e+00, 0.0000e+00, 0.0000e+00],
       dtype=torch.float64) 
input 2: tensor([ 3.0285e+01,  1.2000e+00,  2.1000e+00,  1.9000e+00,  5.9000e+00,
         1.0000e+00,  1.5000e+00,  7.0000e-01,  4.7000e+00,  2.9000e+00,
         1.6000e+00,  1.2000e+00,  5.0000e-01,  2.0000e-01,  2.4000e+00,
         1.9000e+00,  9.1000e+00, -4.9000e+00,  1.0339e+02,  1.0784e+02,
         2.2610e+01,  1.2620e+01,  1.4150e-01,  8.6309e+01,  6.4900e+01,
         6.7300e-02,  2.3992e+00,  7.9960e-01,  5.7005e+00,  8.0030e-01,
         8.9980e-01,  1.5996e+00,  

In [ ]:
for data, label, in train_loader:
    print(net(data.float()))
    print(label)

In [ ]:
print()

In [352]:
 np.empty((1,0))

array([], shape=(1, 0), dtype=float64)

In [648]:
preds = np.empty((1,0))
for data, target in val_loader:
    
    output = net(data.float())
    preds  = np.append(preds, output.detach().numpy().T)

    
    
preds
    

array([ 0.81842214,  0.4361662 ,  0.91527337, ..., -0.16868234,
        0.06047469,  0.06885672])

In [649]:
pd.DataFrame(preds)

,0
0,0.818422
1,0.436166
2,0.915273
3,-0.379051
4,-0.287050
...,...
2280,0.641530
2281,0.032958
2282,-0.168682
2283,0.060475


## Gather Betting Data

In [359]:
# read betting data
betting_data = pd.read_csv('./data/nba_betting_data_2010_present.csv')
betting_data = betting_data.dropna()
    
def clean_bet_data(df = betting_data):

    
    df['date'] = pd.to_datetime(df['date'])
    df['home_team_abbr'] = df['home_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'
                                                                                        }
                                                                             )
    df['away_team_abbr'] = df['away_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'}
                                                                           )

    teams = df['home_team_abbr'].unique()

    df = df.sort_values(['date'])
    df['rest'] = np.nan
    for team in teams:
        team_data = df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team)]
        team_data['rest'] = (team_data['date'] - team_data['date'].shift(1)) / np.timedelta64(1, 'D')
        df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team), 'rest'] = team_data['rest']


    df['point_diff'] = df['home_score'] - df['away_score']
    df['point_total'] = df['home_score'] + df['away_score']
    df['covered_spread'] = (df['home_score'] + df['spread'] > df['away_score']).astype(int)
    df['over'] = (df['point_total'] > df['total']).astype(int)
    
    df['prev_cover'] = df['covered_spread'].shift(1)
    df['prev2_cover'] = df['covered_spread'].shift(2)
    
    relevant_betting = df[['date', 'home_team_abbr',  'away_team_abbr',
                           'home_score', 'away_score', 'point_diff', 
                           'rest', 'point_total', 'spread', 'total',
                          'covered_spread', 'over', 'prev_cover', 'prev2_cover']]

    return relevant_betting

bet_data_clean = clean_bet_data()

array([ 12, -19,  12, ...,   1,  11,   8], dtype=int64)

In [360]:
bet_data_clean[['']]

,date,home_team_abbr,away_team_abbr,home_score,away_score,point_diff,rest,point_total,spread,total,covered_spread,over,prev_cover,prev2_cover
1012,2011-12-25,NYK,BOS,106,104,2,NaN,210,-5.0,190.5,0,1,NaN,NaN
1013,2011-12-25,DAL,MIA,94,105,-11,NaN,199,4.5,188.5,0,1,0.0,NaN
1014,2011-12-25,LAL,CHI,87,88,-1,NaN,175,4.5,183.5,1,0,0.0,0.0
1015,2011-12-25,OKC,ORL,97,89,8,NaN,186,-7.5,194.5,1,0,1.0,0.0
1016,2011-12-25,GSW,LAC,86,105,-19,NaN,191,6.0,207.5,0,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11935,2021-02-04,LAL,DEN,114,93,21,4.0,207,-5.0,217.0,1,0,1.0,1.0
11934,2021-02-04,MEM,HOU,103,115,-12,1.0,218,-3.5,226.0,0,0,1.0,1.0
11933,2021-02-04,PHI,POR,105,121,-16,1.0,226,-10.0,223.0,0,1,0.0,1.0
11932,2021-02-04,DAL,GSW,116,147,-31,2.0,263,-4.0,229.5,0,1,0.0,0.0
